# Introduction to Automation with LangChain, Generative AI, and Python
**3.3: Pydantic Parser**
* Instructor: [Jeff Heaton](https://youtube.com/@HeatonResearch), WUSTL Center for Analytics and Business Insight (CABI), [Washington University in St. Louis](https://olin.wustl.edu/faculty-and-research/research-centers/center-for-analytics-and-business-insight/index.php)
* For more information visit the [class website](https://github.com/jeffheaton/cabi_genai_automation).

Pydantic is a data validation and settings management library in Python that uses Python type annotations. It's designed to allow for quick and easy data parsing and validation using Python's standard typing system.

Some of the key features of Pydantic:

* Data validation: It validates data to ensure it conforms to the expected format, converting types where necessary.
* Editor support: Pydantic models are classes that leverage Python's type hints, making them easy to use with modern editors that provide features like type checking and autocompletion.
* Error handling: It provides detailed and human-readable error reports to help identify where and why data failed to validate.
* Settings management: Pydantic is often used for managing settings/configurations, making it easy to load parameters from environment variables, JSON files, or other sources.
* Extensible: You can extend models with methods and properties, and use Pydantic's validation decorators to perform custom validation.
* Integration with other libraries: It works well with many other libraries, such as FastAPI for building APIs, enhancing their usability and functionality.

Overall, Pydantic is highly appreciated for its robustness and ease of use in ensuring that data inputs conform to specified formats, making it a valuable tool in modern Python development, especially in web development and data processing applications.

LangChain, a library designed to facilitate the building of applications with language models, offers various tools to manage and enhance interactions with these models. One of these tools is the PydanticOutputParser, which integrates Pydantic's powerful validation capabilities with the output of large language models (LLMs) like GPT.

The primary goal of the PydanticOutputParser is to ensure that the outputs from a language model are structured and adhere to a predefined schema. This is especially important in applications where consistent and reliable data formats are crucial, such as in data extraction tasks, API responses, or any scenario requiring subsequent automated processing of the model's output.

In [1]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_aws import ChatBedrock

MODEL = 'mistral.mistral-7b-instruct-v0:2'
# Initialize bedrock, use built in role
llm = ChatBedrock(
    model_id=MODEL,
    model_kwargs={"temperature": 0.5},
)

The following code uses an LLM to tell a joke. The pydantic parser ensures that the joke ends in a question mark. Such checking ensures that the somewhat random LLM produces output that aligns with our expectations.

In [2]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke about cats."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

Joke(setup='Why was the cat sitting on the computer keyboard?', punchline='He wanted to purr-use the internet!')

In [6]:
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")
    @validator('name')
    def validate_name(cls, value):
        parts = value.split()
        if len(parts) < 2:
            raise ValueError("Name must contain at least two words.")
        if not all(part[0].isupper() for part in parts):
            raise ValueError("Each word in the name must start with a capital letter.")
        return value

actor_query = "Generate the filmography for Julia Roberts."

parser = PydanticOutputParser(pydantic_object=Actor)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": actor_query})

Actor(name='Julia Roberts', film_names=['Pretty Woman', 'Sleeping with the Enemy', 'Mystic Pizza', 'Notting Hill', 'Erin Brockovich', "Ocean's Eleven", 'The Mexican', 'Full Circle', 'Mona Lisa Smile', "Charlie Wilson's War", 'Duplicity', 'Eat Pray Love', 'August: Osage County', 'Money Monster', "Mother's Day"])